In [1]:
import sys
sys.path.insert(0, '../' )

In [2]:
from utils import randombytes as rd

rd.randombytes(16)

b"\x81\x81-\x8b\xea\xb6@'S\x7f\xc6\x81zK\x94L"

In [3]:
from libgf264fft import clib_wrapper as cgf

cgf.gf264_mul( 1 ,1 )

1

In [4]:
from utils import gf2192 as gf

gf.GF_EXT

3

In [5]:
# the commit scheme

from commit import merkeltreecommit as mt


In [6]:
# challenge

from utils import hash as H


In [7]:
def _gf_fft( fi , rho , offset ):
    fi_len = len(fi)
    assert 0==(fi_len&(fi_len-1))
    assert 0==(rho&(rho-1))
    fi_gf264 = [ [ (e>>(64*j))&0xffffffffffffffff for e in fi ] for j in range(gf.GF_EXT) ]
    r_gf264 = [ [] for _ in range(gf.GF_EXT) ]
    for j in range(gf.GF_EXT):
         for k in range(rho):
              r_gf264[j].extend( cgf.btfy( fi_gf264[j] , offset ^ (k*fi_len) ) )
    r = list(r_gf264[0])
    for i in range(len(r)):
        for j in range(1,gf.GF_EXT):
            r[i] |= ((r_gf264[j][i])<<(64*j))
    return r

#### test #######

offset = 16

fi = [ i+(i<<64)+(i<<128) for i in range(7,9) ]

print( "fi" , [hex(i) for i in fi] )

r = _gf_fft( fi , 4 , offset )

print( "r" , [ hex(i) for i in r] )


fi ['0x700000000000000070000000000000007', '0x800000000000000080000000000000008']
r ['0xcb9ea337ae4e8620cb9ea337ae4e8620cb9ea337ae4e8620', '0xcb9ea337ae4e8628cb9ea337ae4e8628cb9ea337ae4e8628', '0x5d717ce9218663005d717ce9218663005d717ce92186630', '0x5d717ce9218663805d717ce9218663805d717ce92186638', '0xc7919d84d4b13af7c7919d84d4b13af7c7919d84d4b13af7', '0xc7919d84d4b13affc7919d84d4b13affc7919d84d4b13aff', '0x9d8297de8e7dae709d8297de8e7dae709d8297de8e7dae7', '0x9d8297de8e7daef09d8297de8e7daef09d8297de8e7daef']


In [8]:
#def _btfy_1( fi , offset ):
#    l = len(fi)
#    fi_gf264 = [ [ (e>>(64*j))&0xffffffffffffffff for e in fi ] for j in range(gf.GF_EXT) ]
#    r_gf264 = [ [] for _ in range(gf.GF_EXT) ]
#    offset_gf264 = cgf.index_to_gf264(offset)
#    for j in range(gf.GF_EXT):
#         for k in range( 0,l,2 ):
#              vk0 = fi_gf264[j][k]^cgf.gf264_mul( fi_gf264[j][k+1] , offset_gf264^cgf.index_to_gf264(k) )
#              vk1 = fi_gf264[j][k+1]^vk0
#              r_gf264[j].extend( [vk0,vk1] )
#    r = list(r_gf264[0])
#    for i in range(len(r)):
#        for j in range(1,gf.GF_EXT):
#            r[i] |= ((r_gf264[j][i])<<(64*j))
#    return r
#
##### test #######
#
#print( "fi", [hex(i) for i in fi] )
#
#fi1 = _btfy_1( fi , offset )
#print( "f1", [hex(i) for i in fi1] )
#
#

In [9]:
def _ibtfy_1( vi , offset ):
    l = len(vi)
    vi_gf264 = [ [ (e>>(64*j))&0xffffffffffffffff for e in vi ] for j in range(gf.GF_EXT) ]
    r_gf264 = [ [] for _ in range(gf.GF_EXT) ]
    offset_gf264 = cgf.index_to_gf264(offset)
    for j in range(gf.GF_EXT):
         for k in range( 0,l,2 ):
              vk1 = vi_gf264[j][k+1]^vi_gf264[j][k]
              vk0 = vi_gf264[j][k]^cgf.gf264_mul( vk1 , offset_gf264^cgf.index_to_gf264(k) )
              r_gf264[j].extend( [vk0,vk1] )
    r = list(r_gf264[0])
    for i in range(len(r)):
        for j in range(1,gf.GF_EXT):
            r[i] |= ((r_gf264[j][i])<<(64*j))
    return r

#### test #######

print( "r0", [hex(i) for i in r] )

r1 = _ibtfy_1( r , offset )
print( "r1", [hex(i) for i in r1] )



r0 ['0xcb9ea337ae4e8620cb9ea337ae4e8620cb9ea337ae4e8620', '0xcb9ea337ae4e8628cb9ea337ae4e8628cb9ea337ae4e8628', '0x5d717ce9218663005d717ce9218663005d717ce92186630', '0x5d717ce9218663805d717ce9218663805d717ce92186638', '0xc7919d84d4b13af7c7919d84d4b13af7c7919d84d4b13af7', '0xc7919d84d4b13affc7919d84d4b13affc7919d84d4b13aff', '0x9d8297de8e7dae709d8297de8e7dae709d8297de8e7dae7', '0x9d8297de8e7daef09d8297de8e7daef09d8297de8e7daef']
r1 ['0x700000000000000070000000000000007', '0x800000000000000080000000000000008', '0x700000000000000070000000000000007', '0x800000000000000080000000000000008', '0x700000000000000070000000000000007', '0x800000000000000080000000000000008', '0x700000000000000070000000000000007', '0x800000000000000080000000000000008']


In [18]:
def _dummy( *params ) : return None

def ldt_gen_proof( f0 , h_state , verbose = 1 ):
    if 1 == verbose : dump = print
    else : dump = _dummy

    poly_len = len(f0)
    assert 0==(poly_len&(poly_len-1)) , 'poly_len is not a power of 2'
    offset = 1<<63

    proof = []
    rt = []

    dump( f"original poly {f0}")
    dump( "\n### commit phase ###" )
    vi = _gf_fft( f0 , 32 , offset )
    dump( f"Assume the evaluate on [{hex(offset)},|{poly_len}|x32) has been committed ?" )
    dump( f"do a redundent commit here for checking correctness" )
    gf_size = gf.GF_EXT * 8
    i = 0
    xi = int.from_bytes( H.gen( h_state , bytes([3+i,1]) )[:gf_size] , byteorder='little' )
    dump( f"derive x0 <- H( h_state || 3 || 1 ): {hex(xi)}" )
    while( 2 < poly_len ):
        dump( f"iteration {i}: [{poly_len}] -> [{poly_len//2}]:" )
        mesg = [ vi[i].to_bytes(gf_size,'little')+vi[i+1].to_bytes(gf_size,'little') for i in range(0,len(vi),2) ]
        rti , ri , mti = mt.commit( mesg )
        rt.append( (rti,mesg,ri,mti) )
        proof.append( rti )
        dump( f"commit evaluated valuse. --> proof: [{len(rt)}]   #mesg: {len(mesg)}" )

        h_state = H.gen( h_state , xi.to_bytes(gf_size,byteorder='little') )
        dump( f"update h_state <- H( h_state|| xi ): {h_state}" )

        i = i+1
        dump( f"update i: {i}" )
        xi = int.from_bytes( H.gen( h_state , bytes([3+i,1]) )[:gf_size] , byteorder='little' )
        dump( f"derive new challenge xi <- H(h_state||{3+i}||1) : {hex(xi)}" )

        dump( f"deriving new polynomial of length [{poly_len//2}]" )
        vi = _ibtfy_1( vi , offset )
        vi_e = vi[::2]
        vi_o = vi[1::2]
        vi = [ vi_e[i]^gf.mul(vi_o[i],xi) for i in range(len(vi_e)) ]
        dump( f"evaluated values generated by 1 stage of ibtfy  [{len(vi)}]" )
        offset = offset >> 1
        poly_len = poly_len//2
    _ibtfy_1( vi , offset )
    dump( [hex(i) for i in vi ] )
    dump( f"commit deg 1 poly: {hex(vi[0])} + x* {hex(vi[1])}" )
    c0, c1 = vi[0].to_bytes(gf_size,'little') , vi[1].to_bytes(gf_size,'little')
    proof.append( (c0,c1) )
    h_state = H.gen( xi.to_bytes(gf_size,byteorder='little') , c0 , c1 )
    dump( f"update h_state <- H( xi || c0 || c1 ): {h_state}" )

    dump( "\n### query phase ###" )
    Nq = 26
    queries = [ H.gen(h_state,bytes( [ 3+i+1 , j ] )) for j in range(1,Nq+1) ]
    idx_mask = (len(f0)*32//2)-1
    queries = [ int.from_bytes(e)&idx_mask for e in queries ]
    dump( f"Queries: [{len(queries)}], {queries}" )

    j = 0
    for rti,mesg,ri,mti in rt :
        dump( f"open iteration: {j}" )
        proof.append( [ mt.open(mesg,queries[i],ri,mti) for i in range(Nq) ] )
        dump( f"proof len:[{len(proof)}] : auth path len:{len(proof[-1][0])}" )
        queries = [ q//2 for q in queries ]
        j = j+1
    return proof


###########  test   #############

proof = ldt_gen_proof( [1,2,3,4] , b'123' )


original poly [1, 2, 3, 4]

### commit phase ###
Assume the evaluate on [0x8000000000000000,|4|x32) has been committed ?
do a redundent commit here for checking correctness
derive x0 <- H( h_state || 3 || 1 ): 0x5b9610dcbdcfa2f0b480abae7e2526ddad75cf20f9becf1b
iteration 0: [4] -> [2]:
commit evaluated valuse. --> proof: [1]   #mesg: 64
update h_state <- H( h_state|| xi ): b'\xf8\xff?\x90]\xfe\x94?jM\n\x0cH\xba\x04nU\xd0\xed\xe5De3r<-n\x9fL\xd2R\xf6'
update i: 1
derive new challenge xi <- H(h_state||4||1) : 0xc2699f897fff266d98801881da3468244cf53838795df1ef
deriving new polynomial of length [2]
evaluated values generated by 1 stage of ibtfy  [64]
['0x252d0f44fe58c298a099933e3b689288462b8be429f6914e', '0x2c8b716101a45b01c299f13953b9322e75ff6b05cc8156ea', '0x665a194919a1e9154f5bfc1aff7b9fe50b760410b50984de', '0x6ffc676ce65d708c2d5b9e1d97aa3f4338a2e4f1507e437a', '0x24fa09d5704bf7365de7b76cae0a6f48c2c590fb8c4c0e0e', '0x2d5c77f08fb76eaf3fe7d56bc6dbcfeef111701a693bc9aa', '0x678d1fd897b2dcbbb2